# 1. Load Library

In [1]:
import os
import numpy as np
import cv2
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, ResNet101, ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# 2. Load Dataset

In [2]:
# Load data dari dataset.h5
with h5py.File("dataset.h5", "r") as hf:
    x_train = hf["train_images"][:]
    y_train = hf["train_labels"][:]
    x_val = hf["val_images"][:]
    y_val = hf["val_labels"][:]

print("Loaded dataset.h5 (tanpa augmentasi)")

Loaded dataset.h5 (tanpa augmentasi)


In [3]:
# Cek len(label_map) ==  y_train.shape[1]
# print("Jumlah kelas dari label_map      :", len(label_map))
print("Jumlah kelas dari y_train.shape[1]:", y_train.shape[1])

Jumlah kelas dari y_train.shape[1]: 8


# 3. Build and Train 3 Model CNN-ResNet

In [4]:
# Build model function
def build_resnet_model(resnet_version, num_classes, learning_rate):
    base_model = resnet_version(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training config
resnet_versions = [ResNet50, ResNet101, ResNet152]
batch_sizes = [16, 32] # 64, 128
learning_rate = 0.01
num_classes = y_train.shape[1]

models = {}

In [ ]:
# Tanpa Augmentasi
for batch_size in batch_sizes:
    for resnet in resnet_versions:
        model = build_resnet_model(resnet, num_classes, learning_rate)
        print(f"Training {resnet.__name__} WITHOUT AUGMENTATION, batch_size={batch_size}")
        history = model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            batch_size=batch_size,
            epochs=10
        )
        filename = f"{resnet.__name__}_noaug_bs{batch_size}.h5"
        model.save(filename)
        models[filename] = model

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 92s 1us/step
Training ResNet50 WITHOUT AUGMENTATION, batch_size=16
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1864s 9s/step - accuracy: 0.1782 - loss: 3.3242 - val_accuracy: 0.1294 - val_loss: 2.4769
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1597s 8s/step - accuracy: 0.3094 - loss: 1.7700 - val_accuracy: 0.1327 - val_loss: 2.6675
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1337s 7s/step - accuracy: 0.3677 - loss: 1.5891 - val_accuracy: 0.1316 - val_loss: 2.9079
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1108s 6s/step - accuracy: 0.4266 - loss: 1.4396 - val_accuracy: 0.2400 - val_loss: 2.3658
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1082s 5s/step - accuracy: 0.4569 - loss: 1.3362 - val_accuracy: 0.1814 - val_loss: 11.0585
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1128s 6s/step - accuracy: 0.4891 - loss: 1.2813 - val_accuracy: 0.2058 - val_loss: 4.7386
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1091s 6s/step - accuracy: 0.5190 - loss: 1.2113 - val_accuracy: 0.2190 - 

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 128s 1us/step
Training ResNet101 WITHOUT AUGMENTATION, batch_size=16
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2291s 11s/step - accuracy: 0.1606 - loss: 3.4419 - val_accuracy: 0.1327 - val_loss: 2.1376
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1970s 10s/step - accuracy: 0.2462 - loss: 1.8738 - val_accuracy: 0.1327 - val_loss: 2.3913
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2013s 10s/step - accuracy: 0.3105 - loss: 1.7362 - val_accuracy: 0.1327 - val_loss: 2.5885
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1924s 10s/step - accuracy: 0.3506 - loss: 1.6434 - val_accuracy: 0.2434 - val_loss: 4.4663
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1973s 10s/step - accuracy: 0.3834 - loss: 1.5515 - val_accuracy: 0.2898 - val_loss: 1.8297
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2019s 10s/step - accuracy: 0.3916 - loss: 1.5195 - val_accuracy: 0.3075 - val_loss: 1.9930
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 1911s 10s/step - accuracy: 0.4443 - loss: 1.4330 - val_accuracy:

234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 106s 0us/step
Training ResNet152 WITHOUT AUGMENTATION, batch_size=16
Epoch 1/10


In [6]:
import os

# Tanpa Augmentasi
for batch_size in batch_sizes:
    for resnet in resnet_versions:
        filename = f"{resnet.__name__}_noaug_bs{batch_size}.h5"
        if os.path.exists(filename):
            print(f"Skipping {filename}, already exists.")
            continue  # Lewati model ini karena sudah dilatih dan disimpan

        model = build_resnet_model(resnet, num_classes, learning_rate)
        print(f"Training {resnet.__name__} WITHOUT AUGMENTATION, batch_size={batch_size}")
        history = model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            batch_size=batch_size,
            epochs=10
        )
        model.save(filename)
        models[filename] = model


Skipping ResNet50_noaug_bs16.h5, already exists.
Skipping ResNet101_noaug_bs16.h5, already exists.
Training ResNet152 WITHOUT AUGMENTATION, batch_size=16
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 3081s 15s/step - accuracy: 0.1375 - loss: 3.4878 - val_accuracy: 0.1316 - val_loss: 2.0774
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2607s 13s/step - accuracy: 0.1524 - loss: 2.0748 - val_accuracy: 0.1350 - val_loss: 2.0793
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2513s 13s/step - accuracy: 0.1299 - loss: 2.0811 - val_accuracy: 0.1173 - val_loss: 2.0837
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2644s 13s/step - accuracy: 0.1575 - loss: 2.0760 - val_accuracy: 0.1527 - val_loss: 2.0594
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2606s 13s/step - accuracy: 0.1479 - loss: 2.0681 - val_accuracy: 0.1648 - val_loss: 2.0572
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2700s 14s/step - accuracy: 0.1662 - loss: 2.0623 - val_accuracy: 0.1692 - val_loss: 2.0440
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2617s 13s/step - acc

Training ResNet50 WITHOUT AUGMENTATION, batch_size=32
Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.1554 - loss: 5.4011 

MemoryError: Unable to allocate 519. MiB for an array with shape (904, 224, 224, 3) and data type float32

In [ ]:
# Tambahkan fungsi ini:
def create_tf_dataset(x, y, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Training loop
for batch_size in batch_sizes:
    for resnet in resnet_versions:
        filename = f"{resnet.__name__}_noaug_bs{batch_size}.h5"
        if os.path.exists(filename):
            print(f"Skipping {filename}, already exists.")
            continue

        model = build_resnet_model(resnet, num_classes, learning_rate)
        print(f"Training {resnet.__name__} WITHOUT AUGMENTATION, batch_size={batch_size}")

        train_dataset = create_tf_dataset(x_train, y_train, batch_size)
        val_dataset = create_tf_dataset(x_val, y_val, batch_size, shuffle=False)

        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=10
        )

        model.save(filename)
        models[filename] = model


Skipping ResNet50_noaug_bs16.h5, already exists.
Skipping ResNet101_noaug_bs16.h5, already exists.
Skipping ResNet152_noaug_bs16.h5, already exists.
Training ResNet50 WITHOUT AUGMENTATION, batch_size=32
Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1399s 14s/step - accuracy: 0.1390 - loss: 4.7336 - val_accuracy: 0.1128 - val_loss: 2214.0581
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1138s 11s/step - accuracy: 0.1802 - loss: 2.0210 - val_accuracy: 0.1338 - val_loss: 2.0803
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1178s 12s/step - accuracy: 0.1678 - loss: 2.0252 - val_accuracy: 0.1162 - val_loss: 2.1379
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1129s 11s/step - accuracy: 0.2120 - loss: 1.9826 - val_accuracy: 0.1162 - val_loss: 2.1959
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1120s 11s/step - accuracy: 0.2190 - loss: 1.9545 - val_accuracy: 0.1449 - val_loss: 2.1636
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 1129s 11s/step - accuracy: 0.2388 - loss: 1.9390 - val_accuracy: 0.1327 - val_loss: 2.2986
Epoch 7/10
99/99 ━━━

Training ResNet101 WITHOUT AUGMENTATION, batch_size=32
Epoch 1/10


In [ ]:
# Tambahkan fungsi ini:
def create_tf_dataset(x, y, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Training loop
for batch_size in batch_sizes:
    for resnet in resnet_versions:
        filename = f"{resnet.__name__}_noaug_bs{batch_size}.h5"
        if os.path.exists(filename):
            print(f"Skipping {filename}, already exists.")
            continue

        model = build_resnet_model(resnet, num_classes, learning_rate)
        print(f"Training {resnet.__name__} WITHOUT AUGMENTATION, batch_size={batch_size}")

        train_dataset = create_tf_dataset(x_train, y_train, batch_size)
        val_dataset = create_tf_dataset(x_val, y_val, batch_size, shuffle=False)

        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=10
        )

        model.save(filename)
        models[filename] = model


Skipping ResNet50_noaug_bs16.h5, already exists.
Skipping ResNet101_noaug_bs16.h5, already exists.
Skipping ResNet152_noaug_bs16.h5, already exists.
Skipping ResNet50_noaug_bs32.h5, already exists.
Training ResNet101 WITHOUT AUGMENTATION, batch_size=32
Epoch 1/10
 1/99 ━━━━━━━━━━━━━━━━━━━━ 4:50:24 178s/step - accuracy: 0.0938 - loss: 2.1724

In [5]:
# Tambahkan fungsi ini:
def create_tf_dataset(x, y, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Training loop
for batch_size in batch_sizes:
    for resnet in resnet_versions:
        filename = f"{resnet.__name__}_noaug_bs{batch_size}.h5"
        if os.path.exists(filename):
            print(f"Skipping {filename}, already exists.")
            continue

        model = build_resnet_model(resnet, num_classes, learning_rate)
        print(f"Training {resnet.__name__} WITHOUT AUGMENTATION, batch_size={batch_size}")

        train_dataset = create_tf_dataset(x_train, y_train, batch_size)
        val_dataset = create_tf_dataset(x_val, y_val, batch_size, shuffle=False)

        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=10
        )

        model.save(filename)
        models[filename] = model


Skipping ResNet50_noaug_bs16.h5, already exists.
Skipping ResNet101_noaug_bs16.h5, already exists.
Skipping ResNet152_noaug_bs16.h5, already exists.
Skipping ResNet50_noaug_bs32.h5, already exists.
Training ResNet101 WITHOUT AUGMENTATION, batch_size=32
Epoch 1/10
99/99 [==============================] - 2308s 22s/step - loss: 2.5517 - accuracy: 0.1435 - val_loss: 2.0793 - val_accuracy: 0.1350
Epoch 2/10
99/99 [==============================] - 2150s 22s/step - loss: 2.0868 - accuracy: 0.1305 - val_loss: 2.0780 - val_accuracy: 0.1350
Epoch 3/10
99/99 [==============================] - 2094s 21s/step - loss: 2.0793 - accuracy: 0.1308 - val_loss: 2.0780 - val_accuracy: 0.1350
Epoch 4/10
99/99 [==============================] - 2100s 21s/step - loss: 2.0801 - accuracy: 0.1315 - val_loss: 2.0867 - val_accuracy: 0.1338
Epoch 5/10
99/99 [==============================] - 2109s 21s/step - loss: 2.0786 - accuracy: 0.1318 - val_loss: 2.0793 - val_accuracy: 0.1294
Epoch 6/10
99/99 [==============